## Preprocessing Data With Pandas

Before we begin, consider these questions:

1. What knowledge do we hope to glean from running an unsupervised learning model on this dataset?
2. What data is available? What type? What is missing? What can be removed?
3. Is the data in a format that can be passed into an unsupervised learning model?
4. Can I quickly hand off this data for others to use?


In [1]:
# Dependencies
import pandas as pd

In [2]:
# Data loading CSV
file_path = "..\Resources\shopping_data.csv"
df_shopping = pd.read_csv(file_path, encoding="ISO-8859-1")
df_shopping.head(5)

,CustomerID,Card Member,Age,Annual Income,Spending Score (1-100)
0,1,Yes,19.0,15000,39.0
1,2,Yes,21.0,15000,81.0
2,3,No,20.0,16000,6.0
3,4,No,23.0,16000,77.0
4,5,No,31.0,17000,40.0


### Data Selection - What data is available?

In [3]:
# Columns
df_shopping.columns

Index(['CustomerID', 'Card Member', 'Age', 'Annual Income',
       'Spending Score (1-100)'],
      dtype='object')

In [4]:
# List dataframe data types
df_shopping.dtypes

CustomerID                  int64
Card Member                object
Age                       float64
Annual Income               int64
Spending Score (1-100)    float64
dtype: object

### What data is missing? 

Pandas has the isnull() method to check for missing values. We’ll loop through each column, check if there are null values, sum them up, and print out a readable total:

In [5]:
# Find null values
for column in df_shopping.columns:
    print(f"Column {column} has {df_shopping[column].isnull().sum()} null values")

Column CustomerID has 0 null values
Column Card Member has 2 null values
Column Age has 2 null values
Column Annual Income has 0 null values
Column Spending Score (1-100) has 1 null values


In [6]:
# Determing if the null value rows can be dropped, since it is a small percentage we will drop them. 
df_shopping.count()

CustomerID                203
Card Member               201
Age                       201
Annual Income             203
Spending Score (1-100)    202
dtype: int64

In [7]:
# Drop null rows
df_shopping = df_shopping.dropna()

In [8]:
# Find any duplicate entries
print(f"Duplicate entries: {df_shopping.duplicated().sum()}")

Duplicate entries: 0


In [9]:
# Remove the CustomerID column, since it doesn't offer any insight into customer shopping habits.
df_shopping.drop(columns=["CustomerID"], inplace=True)
df_shopping.head()

,Card Member,Age,Annual Income,Spending Score (1-100)
0,Yes,19.0,15000,39.0
1,Yes,21.0,15000,81.0
2,No,20.0,16000,6.0
3,No,23.0,16000,77.0
4,No,31.0,17000,40.0


### Data Processing

The next step is to move on from what you (the user) want to get out of your data and on to what the unsupervised model needs out of the data.

Recall that in the data selection step, you, as the user, are exploring the data to see what kind of insights and analysis you might glean. You reviewed the columns available and the data types stored, and determined if there were missing values.

For data processing, the focus is on making sure the data is set up for the unsupervised learning model, which requires the following:

1. Null values are handled.
2. Only numerical data is used.
3. Values are scaled. In other words, data has been manipulated to ensure that the variance between the numbers won’t skew results.

Let’s return again to our list of questions.

Is the data in a format that can be passed into an unsupervised learning model?

In [10]:
# Transform string column. To make sure we can use our string data, we’ll transform our strings of Yes and No from the 
# Card Member column to 1 and 0, respectively, by creating a function that will convert Yes to 1 and anything else to 0.
def change_string(member):
    if member == "Yes":
        return 1
    else:
        return 0
df_shopping["Card Member"] = df_shopping["Card Member"].apply(change_string)
df_shopping.head()

,Card Member,Age,Annual Income,Spending Score (1-100)
0,1,19.0,15000,39.0
1,1,21.0,15000,81.0
2,0,20.0,16000,6.0
3,0,23.0,16000,77.0
4,0,31.0,17000,40.0


In [11]:
# The scale for Annual Income is much larger than all the other values in the dataset. We can adjust this format by 
# dividing by 1,000 to rescale those data points.
df_shopping["Annual Income"] = df_shopping["Annual Income"] / 1000
df_shopping.head()

,Card Member,Age,Annual Income,Spending Score (1-100)
0,1,19.0,15.0,39.0
1,1,21.0,15.0,81.0
2,0,20.0,16.0,6.0
3,0,23.0,16.0,77.0
4,0,31.0,17.0,40.0


In [17]:
df_shopping = df_shopping.rename(columns = {'Card Member':'Card_Member', 'Annual Income':'Annual_Income', 'Spending Score (1-100)':'Spending_Score'})
df_shopping.head()

,Card_Member,Age,Annual_Income,Spending_Score
0,1,19.0,15.0,39.0
1,1,21.0,15.0,81.0
2,0,20.0,16.0,6.0
3,0,23.0,16.0,77.0
4,0,31.0,17.0,40.0


### Data Transformation

Data transformation involves thinking about the future. More times than not, there will be new data coming into your data storage (a place where raw data is stored before being touched), with many people working on different types of data analysis. We want to make sure that whoever wants to use the data in the future can do so.

Can I quickly hand off this data for others to use?

In [18]:
# Saving cleaned data into a convenient CSV file for sharing.
file_path = "..\Resources\shopping_data_cleaned.csv"
df_shopping.to_csv(file_path, index=False)